https://deepctr-doc.readthedocs.io/en/latest/Examples.html#hash-layer-with-pre-defined-key-value-vocabulary

In [ ]:
import os

DIR_DATA = os.path.join(os.environ["HOME"], "workspace/third_party/shenweichen/DeepCTR/examples")

In [ ]:
import pandas as pd
import tensorflow as tf

from tensorflow.python.ops.parsing_ops import FixedLenFeature
from deepctr.estimator import DeepFMEstimator
from deepctr.estimator.inputs import input_fn_tfrecord

In [ ]:
# 1.generate feature_column for linear part and dnn part

sparse_features = ['C' + str(i) for i in range(1, 27)]
dense_features = ['I' + str(i) for i in range(1, 14)]

In [ ]:
dnn_feature_columns = []
linear_feature_columns = []

In [ ]:
for i, feat in enumerate(sparse_features):
    dnn_feature_columns.append(
        tf.feature_column.embedding_column(
            tf.feature_column.categorical_column_with_identity(feat, 1000),
            4
        )
    )
    linear_feature_columns.append(tf.feature_column.categorical_column_with_identity(feat, 1000))

In [ ]:
for feat in dense_features:
    dnn_feature_columns.append(tf.feature_column.numeric_column(feat))
    linear_feature_columns.append(tf.feature_column.numeric_column(feat))

In [ ]:
# 2.generate input data for model
feature_description = {
    k: FixedLenFeature(dtype=tf.int64, shape=1)
    for k in sparse_features
}

feature_description.update({
    k: FixedLenFeature(dtype=tf.float32, shape=1)
    for k in dense_features
})

feature_description['label'] = FixedLenFeature(dtype=tf.float32, shape=1)

In [ ]:
train_model_input = input_fn_tfrecord(
    os.path.join(DIR_DATA, 'criteo_sample.tr.tfrecords'),
    feature_description,
    'label',
    batch_size=256,
    num_epochs=1,
    shuffle_factor=10
)

In [ ]:
test_model_input = input_fn_tfrecord(
    os.path.join(DIR_DATA, 'criteo_sample.te.tfrecords'),
    feature_description,
    'label',
    batch_size=2 ** 14,
    num_epochs=1,
    shuffle_factor=0
)

In [ ]:
# 3.Define Model,train, predict and evaluate
model = DeepFMEstimator(
    linear_feature_columns,
    dnn_feature_columns,
    task='binary',
    config=tf.estimator.RunConfig(tf_random_seed=2021)
)

In [ ]:
history = model.train(train_model_input)

In [ ]:
eval_result = model.evaluate(test_model_input)

In [ ]:
print(eval_result)